In [6]:
# GAILを用いて、cameleon-gymの環境で学習する

import gymnasium as gym
import cameleon_gym_env
import time
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.data import rollout
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy

In [7]:
env = gym.make('Cameleon-v0')

passing_point: 8.916346191766797,2.376806810996211
passing_point: 4.865739516674544,6.293706829143909
passing_point: 13.575627103550296,6.277799581434567
passing_point: 3.788724876797044,7.392007315877463
passing_point: 9.82777126231594,13.978396465049464
passing_point: 12.769227116458922,22.988514957858968
passing_point: 13.695401540870199,32.739884965988466
passing_point: 5.859980093190499,29.780565739669633
passing_point: 11.370623891960143,35.887510417732145
passing_point: 20.338087687712495,35.488777277256006
passing_point: 29.257061919759813,32.00211503334013
passing_point: 36.91344776035726,35.96215972753833
passing_point: 32.06300570575794,42.66912029110408
passing_point: 31.04537837961909,33.875658350651584
passing_point: 22.495169944191915,30.11699684672878
passing_point: 30.792887121950884,32.51512833176769
passing_point: 25.945278594942142,24.59044467690208
passing_point: 23.93975679464437,16.524005532376606
passing_point: 17.13298628949625,21.338454055222662
passing_point:

In [10]:
import pickle


SEED = 42

env = make_vec_env(
    "Cameleon-v0",
    rng=np.random.default_rng(SEED),
    n_envs=8,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
env_file_path = env.__module__
print(env_file_path)

path_file = "/media/gaia-22/ESD-ISO/auto_cosmos/cameleon/data/pkl/rollout01.pkl"
rollouts = pickle.load(open(path_file, 'rb'))
 
learner = PPO(
    env=env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0004,
    gamma=0.95,
    n_epochs=5,
    seed=SEED,
)
reward_net = BasicRewardNet(
    observation_space=env.observation_space,
    action_space=env.action_space,
    normalize_input_layer=RunningNorm,
)
gail_trainer = GAIL(
    demonstrations=rollouts,
    demo_batch_size=64,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=8,
    venv=env,
    gen_algo=learner,
    reward_net=reward_net,
)

env.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True
)
gail_trainer.train(200_000)

env.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True
)

print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)


passing_point: 6.048764742677123,5.66405252084599
passing_point: 1.3996269545252238,0.6855237766230209
passing_point: 11.027568024517503,1.9137801139484405
passing_point: 12.2194224150101,6.525338307171868
passing_point: 3.882442266036511,9.456507167448098
passing_point: 13.756327879316135,10.06706416479191
passing_point: 22.773863050763097,8.737860555987703
passing_point: 22.45613164236913,18.52907119789458
passing_point: 28.705106558932542,12.95402859351766
passing_point: 21.02029886711952,7.990706598911682
passing_point: 13.11898587409151,5.003228122904442
passing_point: 9.307530423670272,14.050920796805777
passing_point: 7.246887869175039,5.962381797004117
passing_point: 13.221131398909051,11.889141528313163
passing_point: 17.1495103569893,4.87353540482208
passing_point: 25.273844400013985,4.985295816173869
passing_point: 16.786606570039687,7.050866213703314
passing_point: 20.35077500719296,15.871100599282794
passing_point: 29.476381518875854,19.951702574105134
passing_point: 25.54

TypeError: CameleonEnv.reset() got an unexpected keyword argument 'seed'